In [30]:
import requests 
import bs4
import re
import pandas as pd
import urllib.request
import json
import requests
import re
import os
import random
import urllib

import MySQLdb
db = MySQLdb.connect(host = 'localhost',#本地数据库
                             user = 'root', #用户名
                             passwd = 'yourpw', #数据库密码
                             db = 'mydict', #数据库名
                             charset = 'utf8') 

try:
    with open('data.json', 'r') as f:
        word_base = json.load(f)
except:
    word_base = dict()
NN = 1

def look_up(word):
    url = 'https://cn.bing.com/dict/search?q='+word

    da = requests.request(url=url,method='get')
    soup = bs4.BeautifulSoup(da.text)
    rubbish = [s.extract() for s in soup('script')]
    soup = soup.body
    
    try:
        qdef = soup.findAll('ul')[1]
        quick_def = ''
        for s in qdef.findAll('li'):
            quick_def = quick_def + s.text + '\n'
        quick_def = quick_def.replace('网络','Web.')
    except:
        return None
    
    try:
        p = soup.findAll('div', {"class": "hd_prUS"})[0].text
        p = re.findall(r'\[(.*?)\]', p)[0]
    except:
        p = ''
    
    try:
        s_url = soup.find('div', {"class": "hd_tf"}).a['onclick']
        s_url = s_url.replace("javascript:BilingualDict.Click(this,'",'')
        s_url = s_url.replace("','akicon.png',false,'dictionaryvoiceid')",'')
    except:
        s_url = ''
    
    try:
        more_def = ''
        for pos in soup.findAll('div', {"class": "li_pos"}):
            w_pos = pos.find('div', {"class": "pos"}).text
            for w_def in pos.findAll('div', {"class": "de_co"}):
                more_def = more_def +w_pos+w_def.text+ '\n'
    except:
        return None

    return {'word':word,'sound':p,'chinese':quick_def,'more_def':more_def,'sound_url':s_url}

def get_definition(w):
    global NN
    if(w in word_base):
        da = word_base[w]
    else:   
        da = look_up(w)
        NN = NN+1
        if da != None:
            word_base[w] = da
        else:
            return None
    return da

def insert_res(w):
    sql = 'insert ignore into mydict.dicts values("'+w["word"]+'","'+w["sound"]+'","'+w['more_def']+'","'+w["chinese"]+'")'
    cursor = db.cursor()
    cursor.execute(sql)
    db.commit()   

    if os.path.exists('audio/'+w['word']+'.mp3'):
        return
    else:
        try:
            url = w['sound_url']
            urllib.request.urlretrieve(url,'audio/'+w['word']+'.mp3')
        except:
            return 

In [31]:

def ListToTable(a):
    a = a.replace('*','')
    words = []
    for aa in a.split(' '):
        if aa not in {'','\n'}:
            words.append(aa)

    

    
    hhd = 1     
    for word in words:
        w = get_definition(word)
        if NN % 10 == 0:
            with open('data.json', 'w') as f:
                json.dump(word_base, f)
        if w != None:
            try:
                insert_res(w)
            except:
                pass
            print('已完成第'+str(hhd)+'个:  '+w['word'])
            hhd = hhd+1  

    with open('data.json', 'w') as f:
        json.dump(word_base, f)

    print('ALL DONE!!!\n')
    res = '|||\n|---|---|\n'
    for w in words:
        ch = word_base[w]['chinese'].strip().replace('\n','<br>')
        res = res+ '|**' + w + '**|' + ch + '|\n'
 
    return res

In [37]:
my_f = open('26-30.md','r',encoding='utf8').readlines()
word_s = []
sign = 0
for s in my_f:
    if '## Section' in s:
        sign = 1
    elif s.strip() != '' and sign == 1:
        sign = 0
        word_s.append(s)

In [33]:

hh = [ListToTable(s) for s in word_s]

In [41]:
fin = open('26-30.md','r',encoding='utf8').read()
for i in range(len(hh)):
    fin = fin.replace(word_s[i],hh[i])
f = open('26-30.md','w',encoding='utf8')
f.write(fin)
f.close()
    


"## Section 26 median\n\n* unidimensional transcendent aver avow pugnacious deferential embargoe treatise  spate counterforce stimulant concomitant futility sloth elusive paraphernalia gleaming circumstellar churn disk\n \n\n\n1. Some novelists immodestly idealized and exaggerated the significance of their work, but others, **declining** to exalt the role of the writer, question a **transcendent** view of the art. \n   * decline有拒绝的意思 to refuse\n   * 一些小说家会毫不客气的理想化及夸张他们工作的重要意义，但其他小说家拒绝拔高作者的地位，觉得艺术创作也没什么了不起的。\n   * question在这里相当于decline的意思，但transcendent不认识也就无所谓了，其他小说家question 那种认为 art 是transcendent的view\n   * transcenden 的意思是 greater, better, more important, or going past or above all others 超常的；杰出的，卓越的；极其伟大的\n4. The trade in scientific literature in nineteenth-century Germany was so robust that publisher constantly worried about **supplies**  of new titles, an anxiety that gave even relatively undistinguished authors, who made their living writing technical treatises, **strong bargaini